<a href="https://colab.research.google.com/github/sainithin12345/Nithin-Crop-Yield-full-website-code-/blob/main/NithinPlantDisease_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from google.colab import drive, files
import zipfile
import os

In [2]:
# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
# Extract dataset from Google Drive
zip_file_path = '/content/drive/MyDrive/archive (5).zip'
extracted_folder_path = '/content/plant_disease_dataset'

if os.path.exists(zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_folder_path)


In [11]:
# Set dataset paths
train_data_dir = '/content/plant_disease_dataset'
validation_data_dir = '/content/plant_disease_dataset'


In [21]:
# Set constants
img_width, img_height = 224, 224
batch_size = 16
epochs = 3

In [22]:
# Data augmentation and normalization
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(validation_data_dir,
                                                        target_size=(img_width, img_height),
                                                        batch_size=batch_size,
                                                        class_mode='binary')

Found 41276 images belonging to 2 classes.
Found 41276 images belonging to 2 classes.


In [23]:
# Define the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


In [24]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [25]:
# Train the model
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // batch_size,
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.samples // batch_size)

Epoch 1/3
2579/2579 [==============================] - 7385s 3s/step - loss: 0.6940 - accuracy: 0.4984 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/3
2579/2579 [==============================] - 7148s 3s/step - loss: 0.6932 - accuracy: 0.4973 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 3/3
2579/2579 [==============================] - 7169s 3s/step - loss: 0.6933 - accuracy: 0.4988 - val_loss: 0.6932 - val_accuracy: 0.5000


In [32]:
# Provide option to choose image file for prediction
print("Please upload an image file for prediction:")
uploaded = files.upload()


Please upload an image file for prediction:


Saving 1a1184f8-c414-4ead-a4c4-41ae78e29a82___RS_HL 1971.JPG to 1a1184f8-c414-4ead-a4c4-41ae78e29a82___RS_HL 1971.JPG


In [33]:
for filename in uploaded.keys():
    img = tf.keras.preprocessing.image.load_img(filename, target_size=(img_width, img_height))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) / 255.0

    # Predict class
    result = model.predict(img_array)
    if result[0] < 0.5:
        print("Prediction: Non-Diseased")
    else:
        print("Prediction: Diseased")

1/1 [==============================] - 0s 57ms/step
Prediction: Diseased
